In [1]:
import torch
import torch.nn as nn
import sys
import os

In [2]:
# Device configuration
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda', index=0)

In [4]:
path = "./runs/2_32_100_1e-3_1e-8_Adam_0_25_3class"

In [5]:
decided_model = f"{path}/best-model.pt" #Changed to selected model

In [6]:
from model import *

In [7]:
model = ResNet18(out_dim=3)

/kuacc/users/scansiz22/.conda/envs/comp541/lib/python3.7/site-packages/torchvision/models/_utils.py:209: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and will be removed in 0.15, please use 'weights' instead.
  f"The parameter '{pretrained_param}' is deprecated since 0.13 and will be removed in 0.15, "
/kuacc/users/scansiz22/.conda/envs/comp541/lib/python3.7/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and will be removed in 0.15. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


In [8]:
model.load_state_dict(torch.load(decided_model))
model.to(device)
model.eval()

ResNet18(
  (model): ResNet(
    (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (layer1): Sequential(
      (0): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
      (1): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_runn

In [9]:
from data import create_splits
from torch.utils.data import DataLoader
import pandas as pd
import numpy as np

from sklearn.metrics import classification_report

In [10]:
train_set, val_set, test_set = create_splits(dataset_type="5class")
set(test_set.img_labels)

Prev train example count: 791, cur train example count: 791, cur val example count: 0
Image dir: 718, Image labels: 718
Image dir: 0, Image labels: 0
Image dir: 703, Image labels: 703


{0, 1, 2, 3, 4}

In [11]:
train_set, val_set, test_set = create_splits(dataset_type="5class")

#Conversion Part
conversion = lambda x: 1 if ((x>=1) & (x<4)) else 2 if (x==4) else 0
test_set.img_labels = [conversion(i) for i in test_set.img_labels]

Prev train example count: 791, cur train example count: 791, cur val example count: 0
Image dir: 718, Image labels: 718
Image dir: 0, Image labels: 0
Image dir: 703, Image labels: 703


In [12]:
test_loader = DataLoader(test_set, batch_size=1, shuffle=False, num_workers=1)

In [13]:
np.unique(np.array(test_set.img_labels))

array([0, 1, 2])

In [14]:
def test(device, model, criterion, test_loader, savedir):
    cur_test_loss = 0
    test_tot, test_corr = 0, 0
    model.eval()

    all_preds, all_labels, all_indices = [], [], []
    with torch.no_grad():
        for images, labels, index in test_loader:
            images, labels = images.to(device), labels.to(device)
            outs = model(images)
            test_loss = criterion(outs, labels)
            cur_test_loss += test_loss.item()

            _, preds = torch.max(outs.data, 1)
            test_tot += labels.size(0)
            test_corr += (preds == labels).sum().item()

            preds = preds.cpu().tolist()
            labels = labels.cpu().tolist()
            index = index.tolist()

            all_preds.extend(preds)
            all_labels.extend(labels)
            all_indices.extend(index)
            
    test_loss = cur_test_loss / len(test_loader)
    test_acc = test_corr / test_tot

    print(f'Test loss: {test_loss:.3f}, Test accuracy: {test_acc:.3f}')

    #Print predictions by class
    confusion_matrix = pd.crosstab(np.array(all_preds), np.array(all_labels), rownames=["Predictions"], colnames=["Actuals"]) 
    print(confusion_matrix)

    #Accuracy Results
    classification_res = classification_report(all_labels, all_preds, digits=3)
    print(classification_res)


In [15]:
test(device, model, nn.CrossEntropyLoss(), test_loader, None)

Test loss: 0.111, Test accuracy: 0.957
Actuals        0    1   2
Predictions              
0            178    1   0
1              0  413  23
2              0    6  82
              precision    recall  f1-score   support

           0      0.994     1.000     0.997       178
           1      0.947     0.983     0.965       420
           2      0.932     0.781     0.850       105

    accuracy                          0.957       703
   macro avg      0.958     0.921     0.937       703
weighted avg      0.957     0.957     0.956       703

